In [1]:
# Si se quiere ejecutar el código hay que instalar selenium y tener chromedriver en el path
# pip install selenium

In [1]:
# pip install selenium

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import time
import math
import json

In [6]:
driver = webdriver.Chrome()
driver.get('https://www.newtral.es/fakes/')

In [7]:
button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
button.click()

In [9]:
num_bulos = int(driver.find_element(By.CLASS_NAME, 'results-count').text[1:-1])
num_clicks = math.floor(num_bulos / 15)
(num_bulos, num_clicks)

(1641, 109)

In [10]:
button = driver.find_element(By.ID, 'scroll-load-more')
delay = 5
for i in range(num_clicks):       
    try:
        # And press the button
        button.click()
        
        time.sleep(delay)
    except:
        print('Carga completada')
        elems = driver.find_elements(By.CLASS_NAME, 'quote_link.js-card')
        print(str((len(elems) - 2) / 2) + ' bulos cargados')

In [11]:
elems = driver.find_elements(By.CLASS_NAME, 'quote_link.js-card')
(len(elems) - 2) / 2

1641.0

In [12]:
bulos = {}

In [29]:
import urllib
from bs4 import BeautifulSoup
import requests
import numpy
import os
import re
from lxml.html.clean import clean_html
import datetime
from unicodedata import normalize
import unidecode 

In [27]:
pip install Unidecode

Note: you may need to restart the kernel to use updated packages.


In [34]:
cont = 0
for i in range(2, len(elems), 2):
    
    try:
        title = elems[i].text
        link = elems[i].get_attribute('href')
        
        html = requests.get(link, allow_redirects=False).text
        soup = BeautifulSoup(html, 'html.parser')
        
         # Obtener la fecha de la noticia
        inicio = len(link) - 9
        fin = len(link) -1
        date = link[inicio: fin]
        formated_date =  datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y/%m/%d')
        
        words = dict()
        t = unidecode.unidecode(title)
        title_list = re.sub(r'[^\w\s]', '', t.lower()).split(" ")
        
        for w in title_list:
            if w in words:
                words[w] += 1
            else:
                words[w] = 1
                
        new_text = soup.find('div',attrs={'class' : "o-section__row c-article__content"})    
        for text in new_text('p'):
            s= unidecode.unidecode(text.get_text())
           
            for w in re.sub(r'[^\w\s]', '', s.lower().replace(u'\xa0', u' ').replace(u'\n', u' ')).split(" "):
                if w in words:    
                    words[w] += 1
                else:
                    words[w] = 1
        bulos[cont] = {"titulo" : title, "link" : link, "date" : formated_date, "words_count" : words, "date" : formated_date}
        cont += 1
        
    except:
        print("No se ha podido obtener la informacion de la noticia", i)

No se ha podido obtener la informacion de la noticia 2268


In [2]:
 # bulos

In [36]:
len(bulos)

1640

In [37]:
with open('fakenewsNewtral.json', 'w') as fp:
    json.dump(bulos, fp)